In [1]:
# !pip install paramiko
# !pip install scp

In [2]:
import numpy as np
import time
import datetime
import sys
from CloudComputing import CloudComputingApp
np.set_printoptions(threshold=sys.maxsize)

In [3]:
cloudComputing = CloudComputingApp(instance_size=8)

In [4]:
def task1(array_size, chunk_size, operation='addition'):
        print(f'Generating {array_size}x{array_size} matrix')
        arr = cloudComputing.generate_array(array_size, array_size)
        arr1 = cloudComputing.generate_array(array_size, array_size)
        print(f'Splitting {array_size}x{array_size} matrix into {chunk_size}x{chunk_size}')
        s_arr = cloudComputing.split_row(arr, chunk_size, chunk_size)
        s_arr1 = cloudComputing.split_row(arr1, chunk_size, chunk_size)
        print(f'Computing manual addition of {array_size}x{array_size} matrix')
        res1 = np.array(cloudComputing.matrix_add(arr, arr1))
        print(f'Manual computation done for {operation}')
        print(f"\nComputing distributed {operation} operation for {array_size}x{array_size}")
        # cloudComputing.get_queue(cloudComputing.sqs_client, )
        cloudComputing.compute_matrix_operation(operation, s_arr, s_arr1)
        # merge all the queue results
        print(f"\nMerging computation results")
        res2 = cloudComputing.merge_queue_result(s_arr, array_size, chunk_size)
        # cloudComputing.write_result_to_file(res2, f'{operation}_result.txt')
        #validating result
        print(f"\nVerifying computation results")
        if np.array_equal(res1, res2):
            print(f"Verification Successful!")
        else:
            print(f"Verification Failed!")

In [5]:
def task2(array_size, chunk_size, operation='multiplication'):
        print(f'Generating {array_size}x{array_size} matrix')
        arr = cloudComputing.generate_array(array_size, array_size)
        arr1 = cloudComputing.generate_array(array_size, array_size)
        print(f'Splitting {array_size}x{array_size} matrix into {chunk_size}x{chunk_size}')
        s_arr = cloudComputing.split_row(arr, chunk_size, chunk_size)
        s_arr1 = cloudComputing.split_row(arr1, chunk_size, chunk_size)
        print(f'Computing manual {operation} of {array_size}x{array_size} matrix')
        res1 = np.array(cloudComputing.matrix_dot_product(arr, arr1))
        print(f"\nComputing distribution {operation} operation for {array_size}x{array_size}")
        cloudComputing.compute_matrix_operation(operation, s_arr, s_arr1)
        # wait for all the computation to finish
        time.sleep(cloudComputing.get_wait_time_based_on_matrix_size(array_size))
        # merge all the queue results
        print(f"\nMerging computation results")
        res2 = cloudComputing.merge_queue_result(s_arr, array_size, chunk_size)
        print(f"\nVerifying computation results")
        # cloudComputing.write_result_to_file(res2, f'{operation}_result.txt')
        #validating result
        print(f"\nVerifying computation results")
        if np.array_equal(res1, res2):
            print(f"Verification Successful!")
        else:
            print(f"Verification Failed!")

In [6]:
cloudComputing.prepare_architecture()
# cloudComputing.purge_queue()

Preparing SSH connection
Getting keypairs
Preparing instances
Getting public addresses
['184.72.155.54', '34.227.28.237', '3.92.134.243', '54.161.105.67', '52.87.193.72', '34.239.119.158', '54.85.158.64', '35.174.174.152']
Conencting to Instance-0 with IP Address 184.72.155.54
SSH into the instance: 184.72.155.54
Installing required packages for Instance-0 with IP Address 184.72.155.54
Updating Subscription Management repositories.
Unable to read consumer identity

This system is not registered with an entitlement server. You can use subscription-manager to register.

Red Hat Enterprise Linux 9 for x86_64 - AppStre  21 MB/s |  15 MB     00:00    
Red Hat Enterprise Linux 9 for x86_64 - BaseOS   13 MB/s | 6.9 MB     00:00    
Red Hat Enterprise Linux 9 Client Configuration  16 kB/s | 2.0 kB     00:00    
Dependencies resolved.
 Package        Arch      Version           Repository                     Size
Installing:
 python3-pip    noarch    21.2.3-6.el9      rhel-9-appstream-rhui-rpms

# TASK 1: Addition

In [6]:
start_time = datetime.datetime.now()
task1(array_size=5000, chunk_size=250, operation="addition")
print(f"Total computation time is {(datetime.datetime.now() - start_time)}")

Generating 5000x5000 matrix
Splitting 5000x5000 matrix into 250x250
Computing manual addition of 5000x5000 matrix
Computation time 0:00:04.061402
Manual computation done for addition

Computing distributed addition operation for 5000x5000
Processing Queue 0 with 50 tasks
Processing Queue 1 with 50 tasks
Processing Queue 2 with 50 tasks
Processing Queue 3 with 50 tasks
Processing Queue 4 with 50 tasks
Processing Queue 5 with 50 tasks
Processing Queue 6 with 50 tasks
Processing Queue 7 with 50 tasks

Merging computation results
Received 50 results from queue0
Received 50 results from queue1
Received 50 results from queue2
Received 50 results from queue3
Received 50 results from queue4
Received 50 results from queue5
Received 50 results from queue6
Received 50 results from queue7

Verifying computation results
Verification Successful!
Total computation time is 0:30:27.894985


# TASK 2: Multiplication

In [9]:
start_time = datetime.datetime.now()
task2(array_size=1000, chunk_size=200, operation="multiplication")
print(f"Total computation time is {(datetime.datetime.now() - start_time)}")

Generating 1000x1000 matrix
Splitting 1000x1000 matrix into 200x200
Computing manual multiplication of 1000x1000 matrix
Computation time 0:00:00.163602

Computing distribution multiplication operation for 1000x1000
Queue 0 has 13 tasks
Queue 1 has 12 tasks

Merging computation results
Received 13 results from queue0
Received 12 results from queue1

Verifying computation results

Verifying computation results
Verification Successful!
Total computation time is 0:00:41.980934


## Teardown Infrastructure

In [7]:
cloudComputing.teardown_infrastructure()

Terminating i-06901437f537159ae
Terminating i-01dfb2bedc1d5c000
Terminating i-058a4f3192eae4113
Terminating i-0d8ef0a95c289f9bd
Terminating i-032229429c297509c
Terminating i-01358e58ff58f4247
Terminating i-023708986c8f9ad0f
Terminating i-0c9d53c8a07207f14
https://sqs.us-east-1.amazonaws.com/469282757936/queue0 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/queue1 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/queue2 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/queue3 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/queue4 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/queue5 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/queue6 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/queue7 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/result-queue-0 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/result-queue-1 deleted!
https://sqs.us-east-1.amazonaws.com/469282757936/result-queue-2 deleted!
https

True